In [2]:
import torch
import torchvision
import torch.nn as nn
from torchvision import datasets, models, transforms
from torch.utils.data.sampler import SubsetRandomSampler
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import copy

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('using GPU' if device else 'using CPU')

using GPU


In [ ]:
## dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5), (0.5),)])
train_set = datasets.FashionMNIST('/data/mnist', download=True, train=True, transform=transform)
test_set = datasets.FashionMNIST('/data/mnist', download=True, train=False, transform=transform)

# inja validation va train data ro joda mikonim:
indices = list(range(len(train_set)))
np.random.shuffle(indices)
split = int(np.floor(0.2 * len(train_set)))
train_sample = SubsetRandomSampler(indices[:split])
val_sample = SubsetRandomSampler(indices[split:])
# hala data loader:
train_loader = torch.utils.data.DataLoader(train_set, sampler=train_sample, batch_size = 64)
val_loader = torch.utils.data.DataLoader(train_set, sampler=val_sample, batch_size = 64)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=64, shuffle=True)